# 7. Global optimization: Branch and bound

We can extend the branch-and-prune algorithm to solve another fundamental numerical problem: **global optimization**: for a function $f: \mathbb{R}^n \to \mathbb{R}$,

> minimize f(x) over $x \in X \subseteq \mathbb{R}^n$

We would like to have both the global minimum *value* that the function takes, as well as the **global minimiz*ers***, i.e. the locations where it takes that value.

This is a very difficult problem for general nonlinear, non-convex functions $f$. Interval methods provide one of the only methods for guaranteeing that you have found the true global minimum.

## (Spatial) branch and bound

The basic mechanism of the **branch-and-bound* algorithm is very similar to that of branch-and-prune: we repeatedly bisect and analyse each box.

Now, however, we do not have such a simple condition to check as "does $f(X)$ contain $0$?". We need to think a bit more.

Suppose that we evaluate $f$ at a *point* $x$. Then clearly the global minimum value $m^*$ must satisfy $m^* \le f(x)$.
In other words, $f(x)$ is an *upper bound* for the global minimum $m^*$. [We need to evaluate $f(x)$ using interval arithmetic to bound rounding errors.]  We will keep a running upper bound $m$ which we know satisfies $m^* < m$. For example, if we find an $x$ for which $f(x) < m$ then we set $m := f(x)$. [Strictly, $m := \sup(f(x..x))$.]
    
Now think about a box $X$ in the branching process. If we evaluate $f(X)$ and find that the result lies strictly *above* $m$ then $\text{range}(f, X)$ also lies above $m$, so that $f(x) > m \ge m^*$ for all $x \in X$. Thus $X$ *cannot* contain a global minimizer, and can be discarded.

#### Exercise

1. Implement this.


2. Apply it to the function $f(x) = (x^2 - 2)^2$.

## Using a priority queue

For interval optimization, much more than for root finding, the order in which boxes are dealt with turns out to be very important: the faster we can find candidate points $x$ with lower values of $f(x)$, the more boxes we can exclude, or "fathom", quicker. 

One popular choice is to order the boxes by the infimum (lower bound) of $f(X)$, the heuristic idea being that a box with a lower minimum value of $f(X)$ is more likely to contain a deeper global minimum.
Many papers have explored alternative heuristics.

A suitable data structure to order boxes in this way is `PriorityQueue` from `DataStructures.jl`.

## IntervalOptimisation.jl package

[Note the `s` in the package name, corresponding to British spelling. We may want to change this in the future.]

The `IntervalOptimisation.jl` package contains an implementation of the above algorithm, using a priority queue.

In [ ]:
using IntervalOptimisation

#### Exercise

1. Use the function `minimize` to find the global minimum of $f(x) = (x^2 - 2)^2$.


2. What does it return?


3. Minimize the Himmelblau function,

$$f(x, y) = (x^2+y-11)^2 + (x+y^2-7)^2$$

## The cluster effect

The problem with the above algorithm is that it usually ends up generating a *cluster* of boxes around the global minimum (or minima). This is due to the dependency effect which means that interval evaluations of functions give over-estimates; hence the algorithm is unable to exclude boxes. Increasing the tolerance often results in *more* boxes.

A solution to this is to use a "more accurate" inclusion function. (An inclusion function is a function that returns an enclosure of $\text{range}(f; X)$.)

One option is a **mean-value form**. Effectively this is an enclosure of the first-order Taylor expansion of $f$ around the midpoint of $X$:

$$f_{\text{mvf}}(X) = f(m(X)) + f'(X) * (X - m(X))$$

Here, $m(X)$ is again the midpoint of $X$. (Other points may also be used.)

It can be shown that when the size of $X$ goes to $0$, the overestimate of the true range converges faster (quadratically) using the mean-value form compared to the standard interval extension.